In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
import sklearn.metrics 
from math import sqrt
import xgboost as xg
from tqdm import tqdm


In [ ]:
data=pd.read_csv("master_df.csv")

In [ ]:
data

In [ ]:
cell_name=list(data["cell_num"].unique())
print(cell_name)

In [ ]:
data.set_index("cell_num",inplace=True)

In [ ]:
group_data=data.groupby(["cell_num"])

In [ ]:
X=[]
Y=[]
for i in cell_name:
    if(i not in ['b3c7', 'b3c16', 'b3c37', 'b3c44']):
        temp_data=group_data.get_group(i).head(75)
        temp_data=temp_data.drop(["cycle_life"],axis=1)
        x1=[]
        for j in range(len(temp_data)):
            x1.append(list(temp_data.iloc[j]))
        X.append(x1)
        Y.append(int(data.loc[i,"cycle_life"].head(1)))
    
print(sum(Y)/len(Y))
# print(X)
# print(Y)

In [ ]:
print(len(X),len(Y))

In [ ]:
X=np.array(X)
X.shape

In [ ]:
X=np.reshape(X,(len(X),75*7))

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
sc_y = StandardScaler()
# Scale x and y (two scale objects)
X = sc_x.fit_transform(X)
Y = np.array(Y)
Y = sc_y.fit_transform(Y.reshape(-1, 1))

In [ ]:
X_train, X_test, Y_train, Y_test,= train_test_split(X,Y,test_size=0.3,random_state=0)

In [ ]:
print("X_train:",len(X_train))
print("X_test:",len(X_test))
print("Y_train:",len(Y_train))
print("Y_test:",len(Y_test))

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 1000, num = 10)]
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
min_samples_split = [2, 5, 10, 15, 100]


In [ ]:
params = {
    "n_estimators": n_estimators,
    "max_depth": max_depth,
    "learning_rate":[ 0.1,0.01,0.001],
}


In [ ]:
good_params = {'objective': 'reg:squarederror', 'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'enable_categorical': False,
                'gamma': 0, 'gpu_id': -1, 'importance_type': None, 'interaction_constraints': '', 'learning_rate': 0.300000012, 'max_delta_step': 0, 'max_depth': 7,
                'min_child_weight': 1, 'monotone_constraints': '()', 'n_estimators': 100, 'n_jobs': 12, 'num_parallel_tree': 1, 'predictor': 'auto', 'random_state': 0,
                'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 1, 'tree_method': 'exact', 'validate_parameters': 1,
                'verbosity': None}

xgb_r = xg.XGBRegressor(**good_params)


xgb_r.fit(X_train, Y_train)
pred=xgb_r.predict(X_test)


In [ ]:
pred = sc_y.inverse_transform(pred.reshape(-1, 1))
Y_test = sc_y.inverse_transform(Y_test.reshape(-1, 1))
print(Y_test[0:2])
print(pred[0:2])

In [ ]:

mean_absolute_percentage_error=sklearn.metrics.mean_absolute_percentage_error(Y_test,pred)
# mean_absolute_error=sklearn.metrics.mean_absolute_error(Y_test,pred)
# mean_sqaured_error=sklearn.metrics.mean_squared_error(Y_test,pred,squared=True)
# root_mean_sqaured_error=sklearn.metrics.mean_squared_error(Y_test,pred,squared=False)

In [ ]:
print("Resut: ")
print("MAPE:",mean_absolute_percentage_error*100,"%")


In [ ]:
# xgb_r.get_params()
import matplotlib.pyplot as plt
def predvsactual():
    fig = plt.figure(figsize=(12, 5))
    markerline1, stemlines1, baseline1 = plt.stem(Y_test)
    markerline1.set_markerfacecolor('red')
    markerline, stemlines, baseline = plt.stem(pred)
    markerline.set_markerfacecolor('blue')
    plt.legend(['Actual_cycle_life', 'Predicted_cycle_life'])
    plt.xlabel('Cells')
    plt.ylabel('Cycle Life')
    plt.show()

predvsactual()
